# Assessments Answer Key
This answer key is configured such that you should be able to run the code here and see possible approaches to a working solution. For each topic, it will also link further resources, and go into more detail on certain code chunks. It is not meant to be edited. 

Use these answer keys as a guide as needed. Try to work use the context here to work toward an answer before reaching for the solution.

**If you just want to see the answers, they're all tagged with "SOLUTION", CTRL+F your heart out.**

## Setup
The setup code must be run for the solutions to work properly. Review the breakdown of the setup code in the lab notebook for an explanation of each section.

In [ ]:
import torch
from torchvision import transforms
import numpy as np
from PIL import Image
import spacy
from alibi.datasets import load_cats
from alibi.explainers import AnchorImage
from IPython import display
import pandas as pd
from matplotlib import pyplot as plt
from transformers import AutoTokenizer, BertForSequenceClassification
from textattack import Attack, Attacker
from textattack.attack_recipes import DeepWordBugGao2018
from textattack.datasets import Dataset
from art.attacks.evasion import CarliniL2Method, HopSkipJump
from art.estimators.classification import PyTorchClassifier, BlackBoxClassifier

# put the model on a GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

target_model = torch.hub.load('pytorch/vision:v0.10.0', 
                              'mobilenet_v2', 
                              weights='MobileNet_V2_Weights.DEFAULT', 
                              verbose=False)
target_model.train()
target_model.to(device);

# Define the transforms for preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256), 
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
]);

unnormalize = transforms.Normalize(
   mean= [-m/s for m, s in zip([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])],
   std= [1/s for s in [0.229, 0.224, 0.225]]
)

with open("../data/labels.txt", 'r') as f:
    labels = [label.strip() for label in f.readlines()]

img = Image.open("../data/dog.jpg")
img_tensor = preprocess(img).unsqueeze(0)
unnormed_img_tensor = unnormalize(img_tensor).to(device)

## Adversarial Robustness Toolbox (ART)
- [ART Docs](https://adversarial-robustness-toolbox.readthedocs.io/en/latest/)

### SOLUTION Exercise 1
#### Troubleshooting
_Help! I'm seeing..._
- `ConnectionError`: make sure your local model is running! Run `python /dli/4_assessments/score.py` from a terminal.
- `ValueError: pic should be 2/3 dimensional. Got 4 dimensions.`: Did you remember to `unnormalize` the tensor before passing it to the API? If yes, look at the shape of the unnormalized tensor. Take a look at the examples from prior labs. What information from the unnormalized tensor does the model expect?
- "uhhh it's not predicting a dog at all...": Did you remember to `unnormalize` the tensor before passing it to the API?



In [ ]:
import requests
import base64
from PIL import Image
from io import BytesIO

img_hsj = Image.open("../data/dog.jpg")
img_tensor_hsj = preprocess(img_hsj).unsqueeze(0)

# Convert PIL image to bytes
buffer = BytesIO()
img_hsj.save(buffer, format="JPEG")
image_bytes = buffer.getvalue()

# Encode bytes using base64
encoded_image = base64.urlsafe_b64encode(image_bytes).decode('utf-8')

def send_encoded_image(encoded_image,url="http://127.0.0.1:2718/predict"):
    # Payload data for the POST request
    data = {
        'image': encoded_image
    }

    # Send the POST request
    response = requests.post(url, json=data)

    # Check the response status code
    if response.status_code == 200:
        # Request was successful
        return response.json()
    else:
        # Request failed
        print('Error:', response.text)
        return None

You'll need to run the above to the solution to work!

In [ ]:
def predict(x):
    # Convert numpy array into tensor
    torch_tensor = torch.from_numpy(x).to(device)

    # unnormalize the tensor and convert it to a PIL image
    unnormed_img_hsj = unnormalize(torch_tensor).to(device)
    img_hsj = transforms.functional.to_pil_image(unnormed_img_hsj[0])
    
    # Convert PIL image to bytes
    buffer = BytesIO()
    img_hsj.save(buffer, format="JPEG")
    image_bytes = buffer.getvalue()
    encoded_image = base64.urlsafe_b64encode(image_bytes).decode('utf-8')

    # Send the encoded image to the model endpoint
    resp = send_encoded_image(encoded_image)

    import sys
    amax = np.argmax(resp["probs"])
    sys.stdout.write(f'{resp["label"]}\t{resp["probs"][0][amax]}\r')
    return resp['probs']

### SOLUTION: Exercise 2
The important context for this exercise is that, with `nb_classes=2`, ART "expects" the return value of the `predict` function to be formatted `[p(original class), p(anything else)]`. 

With that, think carefully about how you might need to modify the return value of the predict function such that ART can still do the following...
- Quickly identify that it has crossed the decision boundary
- Recognize that it is "moving" in the right direction

In HSJ, remember that ART is attempting to find the optimal image that crosses the decision boundary while still being recognizable as the original image, which is why it doesn't immediately terminate when we've achieved a single misclassification.


In [ ]:
def predict(x):
    # Convert numpy array into tensor
    torch_tensor = torch.from_numpy(x).to(device)

    # unnormalize the tensor and convert it to a PIL image
    unnormed_img_hsj = unnormalize(torch_tensor).to(device)
    img_hsj = transforms.functional.to_pil_image(unnormed_img_hsj[0])
    
    # Convert PIL image to bytes
    buffer = BytesIO()
    img_hsj.save(buffer, format="JPEG")
    image_bytes = buffer.getvalue()
    encoded_image = base64.urlsafe_b64encode(image_bytes).decode('utf-8')

    # Send the encoded image to the model endpoint
    resp = send_encoded_image(encoded_image)

    import sys
    sys.stdout.write(f'{resp["label"]}\t{resp["prob"]}\r')
    if resp['label'] == original_label:
        return [resp['prob'], 0]
    else:
        return [0, resp['prob']]

The important part is this chunk:

```python
if resp['label'] == original_label:
    return [resp['prob'], 0]
else:
    return [0, resp['prob']]
```

_Help me understand..._
- **What is this even doing?**: Remember that ART expects the format of the return value of `predict` to be `[p(original class), p(anything else)]`. If we generate an adversarial example that gets us over the decision boundary, we want to _force_ the probability of the `German shepherd` class to go to zero, and vice versa.
- **Why can't we do this instead?**

```python
if resp["label"] == curLabel:
    probs = [resp['prob'],1-resp['prob']]
else:
    probs = [1-resp['prob'],resp['prob']]
```

(even if you aren't asking yourself this, try it, and see if you can work out why it doesn't work)

Just because we can crossed the decision boundary and therefore found an image where `p(class X) > p(German shepherd)` doesn't mean that `p(class X) > (1 - p(class X))`. `p(class X)` can be very tiny and still be greater than `p(German shepherd)`, but if we use the above approach and `p(class X) = 0.04`, for example, we'd be telling ART that `p(German shepherd) = 0.96` and leave ART feeling like it just can't find any image that crosses the decision boundary. We want ART to be stoked about crossing the decision boundary! 

Just for fun, after you finish this section, go look at the server you're running in the terminal. That's a lot of requests! It's not exactly a covert attack.

## Alibi
There were no exercises for the TextAttack portion, so we'll skip right to the Alibi section.

### Resources
- [Alibi AnchorText Docs](https://docs.seldon.io/projects/alibi/en/latest/api/alibi.explainers.anchors.anchor_text.html)

### Setup 
You'll need to run this for the exercise solution to work.

In [ ]:
from alibi.utils import spacy_model

model = 'en_core_web_md'
spacy_model(model=model)
nlp = spacy.load(model)

tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

def predict(x):
    inputs = tokenizer(x, return_tensors="pt", padding=True)
    with torch.no_grad():
        output = model(**inputs)
    return output.logits.numpy()

### SOLUTION: Exercise 3
1. Create a list that contains more than one text sample

In [ ]:
text = [
    'My dear aunt Sally loved this movie',
    'This film was strange yet oddly charming',
    'A heartwarming story that had me in tears by the end',
    'The special effects were cool, but the story was lacking',
    'Yawn. I could not stay awake.',
    'I would watch this again and again!'
]

2. Make a simple modification to the `explainer` to deal with 0 or 1 (discrete) outputs

In [ ]:
from alibi.explainers import AnchorText
explainer = AnchorText(
    predictor=predict,
    sampling_strategy='unknown',
    nlp=nlp,
    use_proba=False
)

_NOTE: the above might throw a warning, it's safe to ignore_

_Help me understand_...
- **What does the `use_proba` argument change?**

From the docs... "`use_proba` : `bool` - whether to sample according to the predicted words distribution. If set to `False`, the `top_n` words are sampled uniformly at random." In other words, when we set this to `False`, our `explainer` no longer expects the `predict` function to return logits.

Let's make sure this works!

In [ ]:
for t in text:
    pred = predict(text)
    predicted_class_id = pred.argmax().item()
    explanation = explainer.explain(t, threshold=0.95)
    print(f"\n\n\nText: {t}")
    print(f"Anchor: {explanation.anchor}")
    print(f'Precision: {explanation.precision:.2f}\n')
    
    print(f"Examples where anchor applies and model predicts: {predicted_class_id}\n---------------\n")
    print('\n'.join([x for x in explanation.raw['examples'][-1]['covered_true']])) 
    print(f"\nExamples where anchor applies and model predicts: 0\n---------------\n")
    print('\n'.join([x for x in explanation.raw['examples'][-1]['covered_false']]))